# ArcPy Selection Functions

An essential task in GIS is to select a subset of a layer or feature class that
matches certain criteria.

The criteria could be based on the information carried
within the data's **attributes**, e.g., _block groups with more than 1000 population_,
or based on **locational** traits, e.g., _within 2 miles of any school_.

In this section, we will take a look at several functions that are used to selecting
(or querying) features.

## 1. Selection Query

Structured Query Language (**SQL**) is a powerful language used to define one or more criteria that can consist of attributes, operators, and calculations.

```{hint}
SQL reads as S-Q-L or "sequel" by many professional database developers.
```

Whether you're aware or not, as users of ArcGIS Pro, you must have already used it.
Perhaps, in the form of the following approach.

```{image} ../_static/images/selection_query.png
:class: border mb-2
:alt: "selection query"
:scale: 55%
:align: center
```

Query expressions in ArcGIS adhere to standard SQL expressions.
Accordingly, if we turn the "switch" to **SQL**, we will see the
following equivalent expression written in SQL.

```{image} ../_static/images/selection_query_SQL.png
:class: border mb-2
:alt: "selection query SQL"
:scale: 55%
:align: center
```


### 1.1 Writing SQL expression in ArcPy

Without loss of generality, a single SQL expression can be written
in the following pattern:

`<field name> <logic operator> <value>`

Rules to remember when writing SQL expression for ArcPy functions.

1. Query in ArcPy functions is defined using Python `str`.
2. [Field delimiter](https://pro.arcgis.com/en/pro-app/latest/arcpy/get-started/specifying-a-query.htm#GUID-98106084-6ECE-4647-A50D-50EF0B46EC25)
   must be used to specify a **field of an attribute** table.
   For _shapefile_ and _feature class_ (in _file geodatabase_), the field delimiter
   is **double quotes**, i.e., `"<fieldname>"`.
3. _Text_ values must always be enclosed by **single quotes** `'<some text>'`.

```{admonition} History of Geodatabases
:class: seealso

Read this blog about [a brief history](https://www.esri.com/arcgis-blog/products/arcgis-pro/data-management/its-not-personal-its-mobile/)
of the geodatabase and why personal geodatabases are no longer supported in ArcGIS Pro. And, learn what is mobile geodatabase?

- [Personal geodatabase](https://desktop.arcgis.com/en/arcmap/latest/manage-data/administer-file-gdbs/personal-geodatabases.htm)
- [Types of geodatabases (ArcGIS Pro)](https://pro.arcgis.com/en/pro-app/latest/help/data/geodatabases/overview/types-of-geodatabases.htm)
```

Since there are multiple rules to follow when writing SQL expressions,
we must use a combination of methods that we have learned
about defining Python strings, including `" "` (double quotes), `' '` (single quotes),
`""" """` (triple quotes), and `\` (the escape character).

Now, let’s work on two examples using the _zip_boundaries_ feature class.
Specifically, we will write SQL expressions (in Python) to query against
a field containing **numeric values** and another field containing
**text values**.

- numeric value: `POP2010 > 10000`, (zip codes have more than 10,000 people)
- text value: `PO_NAME = Gainesville` (zip codes in the City of Gainesville)

### 1.2 Expression with triple quotes

Strings defined by "triple quotes" is commonly used to define SQL
expressions in ArcPy because this form can accommodate both double quotes
and single quotes.

In [61]:
# numeric value
print(""""POP2010" > 10000""")

"POP2010" > 10000


In [60]:
# text value
print(""""PO_NAME" = 'GAINESVILLE'""")

"PO_NAME" = 'GAINESVILLE'


```{seealso}
- [SQL reference for query expressions used in ArcGIS](http://pro.arcgis.com/en/pro-app/help/mapping/navigation/sql-reference-for-elements-used-in-query-expressions.htm)
- [Specifying a query in Python](http://pro.arcgis.com/en/pro-app/arcpy/get-started/specifying-a-query.htm)
```

### 1.3 Expression with escape character

Remember that we can include double quote and single quote in string
by using the escape character (`\`).

In [62]:
print("\"")
print("\'")

"
'


In [63]:
query_numeric = "\"POP2010\" > 10000"
print(query_numeric)

"POP2010" > 10000


In [64]:
query_text = "\"PO_NAME\" = \'GAINESVILLE\'"
print(query_text)

"PO_NAME" = 'GAINESVILLE'


### 1.4 Expression with `.format`

In [65]:
query_numeric = "{} > {}".format('"POP2010"', 10000)
print(query_numeric)

"POP2010" > 10000


In [66]:
query_text = "{} = {}".format('"PO_NAME"', "'GAINESVILLE'")
print(query_text)

"PO_NAME" = 'GAINESVILLE'


## 2. Define Compound Criteria

Consider the following compound criteria.

- population greater than 10,000 **AND** name equals to Gainesville
- population greater than 10,000 **OR** name equals to Gainesville

We use `AND` and `OR` to connect two expressions together. Note that
all letters in both words are **capitalized**.
❓ Do you still remember how _logical operators_ are written **in Python**?
Are they upper case or lower case?

In [67]:
query_comp = "{} > {} AND {} = {}".format('"POP2010"', 10000,
                                          '"PO_NAME"', "'GAINESVILLE'")
print(query_comp)

"POP2010" > 10000 AND "PO_NAME" = 'GAINESVILLE'


In [68]:
query_comp = "{} > {} OR {} = {}".format('"POP2010"', 10000,
                                         '"PO_NAME"', "'GAINESVILLE'")
print(query_comp)

"POP2010" > 10000 OR "PO_NAME" = 'GAINESVILLE'


## 3. `Select` Function

[`arcpy.analysis.Select`](https://pro.arcgis.com/en/pro-app/latest/tool-reference/analysis/select.htm)
extracts features from an input feature class or input feature layer, typically
using a SQL expression, and **stores**
them in an output feature class, i.e., saves physically on the drive.

Now, let's apply what we have learned about SQL expressions in this function.

In [25]:
import arcpy

In [26]:
gdb_worksp = r"..\data\class_data.gdb"
arcpy.env.workspace = gdb_worksp

In [4]:
zip_fc = "zip_boundaries"

In [27]:
pop_query = """"POP2010" > 10000"""
zip_output = "zip_q1_out"
arcpy.Select_analysis(zip_fc, zip_output, pop_query)

print("{} zip codes selected.".format(arcpy.GetCount_management(zip_output)))

<Result '..\\data\\class_data.gdb\\zipbnd_q1'>

In [28]:
city_query = "\"PO_NAME\" = \'GAINESVILLE\'"
zip_output = "zip_q2_out"
arcpy.analysis.Select(zip_fc, zip_output, city_query)

print("{} zip codes selected.".format(arcpy.GetCount_management(zip_output)))

<Result '..\\data\\class_data.gdb\\zipbnd_q2'>

In [30]:
comp_query = "{} > {} AND {} = {}".format('"POP2010"', 10000,
                                          '"PO_NAME"', "'GAINESVILLE'")
zip_output = "zipbnd_q3_out"
arcpy.analysis.Select(zip_fc, zip_output, comp_query)

print("{} zip codes selected.".format(arcpy.GetCount_management(zip_output)))

<Result '..\\data\\class_data.gdb\\zipbnd_q2'>

## 4. Select by Attributes

[`arcpy.management.SelectLayerByAttribute()`](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm)
adds, updates, or removes a selection based on an attribute query.

In [31]:
zip_fc = "zip_boundaries"
pop_query = """"POP2010" > 10000"""
zip_lyr = arcpy.management.SelectLayerByAttribute(zip_fc,
                                                  "NEW_SELECTION",
                                                  pop_query)
print("{} records selected.".format(arcpy.GetCount_management(zip_lyr)))

```{admonition} Selection on layers

**Select by attribute** and **Select by location** introduced later only
"temporarily" select features from a specified feature class or layer,meaning
they **DO NOT physically** save files on the hard disk.
See [creating and using layer selections](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/creating-and-using-layer-selections.htm).

It is a good idea to assign the output to **a variable**, just like the example
above. Hence, we can reference that selection later in the script.
```

## 5. Select by Location

[`arcpy.management.SelectLayerByLocation`](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm)
selects features based on a spatial relationship to features in another dataset.

Each feature in the Input Features parameter is evaluated against the features
in the Selecting Features parameter.
If the specified Relationship parameter value is met, the input feature is selected.

```{seealso}
All [spatial relationships](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-by-location-graphical-examples.htm)
supported by ArcGIS Pro.
```

In [33]:
blkgrp_fc = "blockgroups"
cntbnd_fc = "county_boundary"
blkgrp_lyr = arcpy.management.SelectLayerByLocation(
    blkgrp_fc, "WITHIN", cntbnd_fc, "", "", ""
)

Note the difference between the two print statements below.
The first one prints the number of features in the feature class.
The second, on the other hand, prints the number of records get selected
in the layer which references that same feature class.

In [34]:
print("{} features selected.".format(
    arcpy.management.GetCount(blkgrp_fc))
)

178


In [35]:
print("{} records selected.".format(
    arcpy.management.GetCount(blkgrp_lyr))
)

142


## 5. Save Layer Selection to a Feature Class

There are two options to save a "temporary" selection to an output layer.

- `arcpy.conversion.FeatureClassToFeatureClass(<path>, <fc name>)`
- `arcpy.management.CopyFeatures(<full path>)`

It is critical to understand how to set output **path** and **name** in each
of the method.

If path not set, the output feature class will be stored in current workspace.

To specify a path use one of the following methods:

- use `"\\"` to concatenate path and name
- use `os.path.join()` function
- define the full path name altogether

In [38]:
output_path = r"..\data\output_data.gdb"
output_name = "bg_within_cnt"

In [39]:
result = arcpy.conversion.FeatureClassToFeatureClass(
    blkgrp_lyr, output_path, output_name
)
print("{}".format(result.getOutput(0)))

<Result '..\\data\\class_data.gdb\\blockgroups_I75_2mi'>

In [40]:
# path not specified
result = arcpy.management.CopyFeatures(blkgrp_lyr)
print("{}".format(result.getOutput(0)))

<Result '..\\data\\class_data.gdb\\blockgroups_Layer5_CopyFeatures'>

```{seealso}
The `Result` object is used to see the output locations.
To learn more about it, see [here](https://pro.arcgis.com/en/pro-app/latest/arcpy/classes/result.htm).
```

The output feature class can be defined using the following ways.
Note that they result in the same value.

In [41]:
output_path + "\\" + output_name

'..\\data\\class_data.gdb\\blockgroups_I75_2mi'

In [42]:
import os
output_name = os.path.join(output_path, output_name)
output_name

'..\\data\\class_data.gdb\\blockgroups_I75_2mi'

In [43]:
output_fc = r'..data\class_data.gdb\bg_within_cnt'
output_fc